<a href="https://colab.research.google.com/github/PhilSad/stable-diffusion-infinite-outpainting-video/blob/staging/notebooks/colab_infinite_outpaint_generate_base_images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Variable setup:

In [2]:
# Choose a new SESSION_NAME
# will create a directory in your gdrive: MyDrive/stable-diffusion-infinite-outpaint/{SESSION_NAME}
# Choose a SESSION_NAME that you did not use before, otherwise the generated files will be replaced

SESSION_NAME = 'dummy' 
N_STEPS = '5'

Insert the prompts you want to use in a file called input.txt:

In [ ]:
%%file input.txt
A dog sitting in a bench park
A cat sitting in a bench park


In [ ]:
#@title Setup

import os

DRIVE_BASE_DIR = "/content/drive/MyDrive/stable-diffusion-infinite-outpaint"
BASE_IMAGE_DIR = f'{DRIVE_BASE_DIR}/{SESSION_NAME}/base_images'
SD_IMAGE_DIR = f'{DRIVE_BASE_DIR}/{SESSION_NAME}/sd_images'

if not os.path.exists(DRIVE_BASE_DIR):
  os.makedirs(DRIVE_BASE_DIR)

!git clone https://github.com/PhilSad/stable-diffusion-infinite-outpainting-video.git
%cd stable-diffusion-infinite-outpainting-video
!pip install -r requirements.txt

In [ ]:
!python infinite_outpaint.py --prompts_path ../input.txt --n_steps {N_STEPS} --downscale_factor 4 --hq_save_folder {BASE_IMAGE_DIR} --sd_save_folder {SD_IMAGE_DIR}


# Next cell is usefull if you are in one of the following cases:
* You have run out of your GPU alocated time when running the outpainting command and want to post process the outpainted images already done to generate the video (don't restart the outpainting step)
* You already have the images that you want to add the zoom effect on and you just want to post process them for the video. You should save them in the folder `SD_IMAGE_DIR`. Their names should be 0.jpeg, 1.jpeg, 2.jpeg, .....

HQ images will be saved in the `BASE_IMAGE_DIR` directory



In [10]:
import glob
from PIL import Image
from super_resolution import generate_base_images_ia

# Load the existing images in a list
images = []
nb_images = len(list(glob.glob(f'{SD_IMAGE_DIR}/*.jpeg')))
for i in range(nb_images):
  img = Image.open(f'{SD_IMAGE_DIR}/{str(i)}.jpeg')
  images.append(img)

# PostProcess 512x512 images list and save them 
for i, base_image in enumerate(generate_base_images_ia(images)):
    base_image.save(f'{BASE_IMAGE_DIR}/{i}.jpeg')